In [37]:
import pandas as pd
import numpy as np

In [38]:
train = pd.read_csv('raw_data/train.csv', sep=',')
test = pd.read_csv('raw_data/test.csv', sep=',')
train.rename(columns={'AnimalID': 'ID'}, inplace=True)
train = train[['ID', 'Name', 'DateTime', 'AnimalType', 'SexuponOutcome', 'AgeuponOutcome', 
               'Breed', 'Color', 'OutcomeType', 'OutcomeSubtype']]
test['OutcomeType']='test'
test['OutcomeSubtype']='test'


train_ids = train['ID']
target = train['OutcomeType']
subtarget = train['OutcomeSubtype']

test_ids = test['ID']

In [39]:
print train.columns
print test.columns

Index([u'ID', u'Name', u'DateTime', u'AnimalType', u'SexuponOutcome',
       u'AgeuponOutcome', u'Breed', u'Color', u'OutcomeType',
       u'OutcomeSubtype'],
      dtype='object')
Index([u'ID', u'Name', u'DateTime', u'AnimalType', u'SexuponOutcome',
       u'AgeuponOutcome', u'Breed', u'Color', u'OutcomeType',
       u'OutcomeSubtype'],
      dtype='object')


In [40]:
# from sklearn.preprocessing import LabelEncoder
# encoder = LabelEncoder()
# train['foo'] = encoder.fit_transform(train['OutcomeSubtype'].fillna(''))
# encoder.classes_

In [41]:
######################       APPLY LOGIC ON TRAIN AND TESTS 
len_train_before = len(train)
len_test_before = len(test)

trainlen = train.shape[0]
dfall = pd.concat((train, test), axis=0, ignore_index=True)
dfall = dfall.drop(['Name'],axis=1)
dfall = dfall.drop(['ID'],axis=1)


In [42]:
dfall['SexuponOutcome'].value_counts(normalize=False)
#Neuter

Neutered Male    14014
Spayed Female    12633
Intact Female     5004
Intact Male       4985
Unknown           1548
Name: SexuponOutcome, dtype: int64

In [43]:
#ajufou bem pouco
dfall['DeSexed'] = 0
dfall.loc[(dfall.SexuponOutcome =='Unknown'), 'DeSexed'] = -1
dfall.loc[(dfall.SexuponOutcome =='Neutered Male'), 'DeSexed'] = 1
dfall.loc[(dfall.SexuponOutcome =='Spayed Female'), 'DeSexed'] = 1


dfall.loc[(dfall.SexuponOutcome =='Spayed Female'), 'Sex'] = 'Female'
dfall.loc[(dfall.SexuponOutcome =='Intact Female'), 'Sex'] = 'Female'
dfall.loc[(dfall.SexuponOutcome =='Unknown'), 'Sex'] = 'Unknown'
dfall.loc[(dfall.SexuponOutcome =='Neutered Male'), 'Sex'] = 'Male'
dfall.loc[(dfall.SexuponOutcome =='Intact Male'), 'Sex'] = 'Male'
dfall = dfall.drop(['SexuponOutcome'],axis=1)

In [44]:
#dfall['Color'].value_counts(normalize=True)

In [45]:
#Function copied from:
#https://www.kaggle.com/andraszsom/shelter-animal-outcomes/age-gender-and-breed-vs-outcome
def age_to_days(item):
    # convert item to list if it is one string
    if type(item) is str:
        item = [item]
    ages_in_days = np.zeros(len(item))
    for i in range(len(item)):
        # check if item[i] is str
        if type(item[i]) is str:
            if 'day' in item[i]:
                ages_in_days[i] = int(item[i].split(' ')[0])
            if 'week' in item[i]:
                ages_in_days[i] = int(item[i].split(' ')[0])*7
            if 'month' in item[i]:
                ages_in_days[i] = int(item[i].split(' ')[0])*30
            if 'year' in item[i]:
                ages_in_days[i] = int(item[i].split(' ')[0])*365    
        else:
            # item[i] is not a string but a nan
            ages_in_days[i] = 0
    return ages_in_days

from datetime import datetime
def convert_date(dt):
    d = datetime.strptime(dt, "%Y-%m-%d %H:%M:%S")
    return d.year, d.month, d.isoweekday() #Return the day of the week as an integer, where Monday is 1 and Sunday is 7

In [46]:
dfall['ageindays'] = age_to_days(dfall['AgeuponOutcome'])
dfall = dfall.drop(['AgeuponOutcome'],axis=1)

In [47]:
dfall["Year"], dfall["Month"], dfall["WeekDay"] = zip(*dfall["DateTime"].map(convert_date))

In [48]:
dfall = dfall.drop(['DateTime'],axis=1)

In [49]:
dfall = dfall.drop(['OutcomeSubtype'],axis=1)

In [50]:
extra_one_hot = ['DeSexed']
for col in dfall.columns:    
    if ((col != 'OutcomeType') and (col != 'OutcomeSubtype') and (dfall[col].dtypes == object or col in extra_one_hot)):
    #if ((col == 'Sex')):    
        print 'Dummying: ',col
        dfall_dummy = pd.get_dummies(dfall[col], prefix='is'+col)
        dfall = dfall.drop([col], axis=1)
        dfall = pd.concat((dfall, dfall_dummy), axis=1)
    else:
        print 'Not touching: ',col

Dummying:  AnimalType
Dummying:  Breed
Dummying:  Color
Not touching:  OutcomeType
Dummying:  DeSexed
Dummying:  Sex
Not touching:  ageindays
Not touching:  Year
Not touching:  Month
Not touching:  WeekDay


In [51]:
train = dfall[:trainlen]
test = dfall[trainlen:]

In [52]:
# #nao ajudou
# print 'dropping cols with unique values'
# to_be_dropped = []
# for col in train.columns:
#     if len(np.unique(train[col])) == 1:
#         to_be_dropped.append(col)
        
# train = train.drop(to_be_dropped,axis=1)
# test = test.drop(to_be_dropped,axis=1)
# print 'Dropped', len(to_be_dropped), 'columns: ',to_be_dropped

In [100]:
train.shape

(26729, 2102)

In [53]:
test = test.drop(['OutcomeType'],axis=1)
#test = test.drop(['OutcomeSubtype'],axis=1)

assert(len_train_before == len(train))
assert(len_test_before == len(test))

In [54]:
features = train [ [col for col in train.columns if col not in ['OutcomeType'] ]].columns

In [55]:
type(features)

pandas.core.index.Index

In [92]:
from xgboost.sklearn import XGBClassifier
model = XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.5,
       gamma=1, learning_rate=0.25, max_delta_step=2, max_depth=4,
       min_child_weight=0, missing=None, n_estimators=40, nthread=-1,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

model = XGBClassifier(objective='multi:softprob', n_estimators=100)

In [98]:
model

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [94]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(train['OutcomeType'].values)
model.fit(train[features], y)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [95]:
#preds = model.predict_proba(test)
preds = model.predict_proba(train[features])

In [97]:
log_loss(y, preds)
#n_estimators=500 #0.81205
#n_estimators=100 #0.85165

0.85165058484980338

In [83]:
preds = model.predict_proba(test)

In [84]:
results = pd.read_csv("raw_data/sample_submission.csv")

results['Adoption'], results['Died'], results['Euthanasia'] = preds[:,0], preds[:,1], preds[:,2]
results['Return_to_owner'], results['Transfer'] =  preds[:,3], preds[:,4]

results.to_csv("submission.csv", index=False)

In [36]:
from sklearn.metrics import log_loss
log_loss(["spam", "ham", "ham", "spam"],  [[0.5,0.6], [1,0], [1,0], [0,1]])


0.15153395089257965

In [102]:
log_loss(["A"],  [[1,0]])

9.9920072216264148e-16